In [1]:
import os
import io
import sys
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
import re
import subprocess
import numpy as np
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

sys.path.append('../src')

In [5]:
root_dir = 'ABAWLogs/'

databases_dir = {'C': 'C'}

stats_path = 'logs/stats.csv'

results = {}
for k, db in databases_dir.items():
    results[db] = []
    exp_lists = os.listdir(os.path.join(root_dir, db))
    exp_lists.sort(key=lambda exp_name: os.path.getmtime(os.path.join(root_dir, db, exp_name)), reverse=True)
    for exp_name in exp_lists:
        df = pd.read_csv(os.path.join(root_dir, db, exp_name, stats_path), sep=';')
        best_model = df.iloc[df['abaw_devel_f1'].idxmax()].to_dict()
        for name, v in best_model.items():
            if 'loss' in name or 'epoch' in name or 'experiment_name' in name:
                continue
            else:
                best_model[name] = v * 100

        results[db].append({**{'experiment_name': exp_name}, **best_model})

for k, db in results.items():
    pd.DataFrame(results[k]).to_csv('{0}_results.csv'.format(k.replace(os.sep, '_')), index=False)